In [1]:
from src.tools import xy, getFrame, hough
from matplotlib import pyplot as plt
from scipy.stats import mode
from math import ceil
import numpy as np
import cv2 as C

EDGE_TOL = 400
RADIUS_RANGE = (1, 20)
SCAN_STEP = 8
REGEN_TOL = 0.1
VIDEO_FILE = './data/A.mp4'

RED = (255, 0, 0)

B_POS = (141, 500)
B_SIZ = 25

In [ ]:
img = getFrame(VIDEO_FILE, 0)

img2 = img.copy()
x0, y0, x1, y1 = xy(B_POS, B_SIZ)
C.rectangle(img2, (y0, x0), (y1, x1), RED, 2)

# plt.imshow(img, cmap='gray')
plt.imshow(img2)
plt.show()

In [ ]:
OFFSET = 2
x0, y0, x1, y1 = xy(B_POS, B_SIZ)
frame = getFrame(VIDEO_FILE, OFFSET)[x0:x1, y0:y1]

circles =  hough(
  C.Canny(frame, EDGE_TOL, EDGE_TOL),
  minDist=8, param1 = 100, param2 = 15,
  minRadius = 1, maxRadius = 20
)[0][0]
cx, cy, R = circles[0], circles[1], ceil(circles[2]) + 1

frame = frame[cy-R:cy+R, cx-R:cx+R]

plt.imshow(frame, cmap='gray')
plt.axis('off')
plt.savefig('ref.png', bbox_inches='tight')
plt.show()

In [ ]:
res = C.matchTemplate(img, frame, C.TM_CCOEFF_NORMED)
# for all matches make a rectangle
threshold = 0.68
loc = np.where(res >= threshold)

cxy = []
for pt in zip(*loc[::-1]):
  x, dx = pt[0], frame.shape[1]
  y, dy = pt[1], frame.shape[0]
  y = round(y/R)*R

  cxy.append((round(x+dx/2), round(y+dy/2)))
  C.rectangle(img, pt, (x+dx, y+dy), RED, 1)
cxy = np.array(cxy)

Y = mode(cxy[:, 1]).mode
cxy = cxy[cxy[:, 1] == Y]
diffs = np.diff(cxy[:, 0], 1)
mean = np.mean(diffs)


plt.imshow(img)
plt.show()

In [ ]:
print(f"With mean diff of {mean:.2f}R @ R={R}")

In [6]:
data_dict = {
  'R': str(R),
  'Y': str(Y),
  'D': str(mean.round(2))
}

with open('./data.txt', 'w') as f:
  from json import dumps
  f.write(str(dumps(data_dict)))

In [7]:
!rm -rf ./in_dir/* ./out_dir/*

In [ ]:
!ffmpeg -hide_banner -loglevel error -i "{VIDEO_FILE}" './in_dir/%03d.png'
!python3 src/extract.py in_dir out_dir {max_Y}
!rm -r ./out_dir/0.png ./out_dir/000.png